In [237]:
# BeautifulSoup, Pandas, and Requests/Splinter
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import requests


In [238]:
# Check for chrome web driver path
!which chromedriver

/usr/local/bin/chromedriver


In [239]:
# Create Browser instance and pass 'chrome' string
def browser():
    executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
    return Browser('chrome', **executable_path, headless=False)
browser = browser()

In [240]:
# Assign news url to variable
news_url= "https://mars.nasa.gov/news"
browser.visit(news_url)
news_html = browser.html
news_soup = BeautifulSoup(news_html, 'lxml')

In [241]:
# Pretty print 
#print(news_soup.prettify())

In [242]:
news_title = news_soup.find_all('div', class_='content_title')
news_title = news_title[1].get_text()
print(news_title)




10.9 Million Names Now Aboard NASA's Perseverance Mars Rover




In [243]:
# NASA Mars News Scraping

In [265]:
news_p = news_soup.find('div', class_='article_teaser_body')
news_p = news_p.get_text()
print(news_p)


AttributeError: 'NoneType' object has no attribute 'get_text'

In [245]:
# Set image browser
image_url= "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)
image_html= browser.html
image_soup = BeautifulSoup(image_html, 'lxml')
# print(image_soup.prettify())


In [246]:
# Set base url 
base_url = 'https://www.jpl.nasa.gov'

# Get featured image partial url
image_url = image_soup.find("a", {"class":"button fancybox"})['data-fancybox-href']

featured_image_url = base_url + image_url
print(featured_image_url)


https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA22949_ip.jpg


In [247]:
# print(base_url)

In [248]:
# Mars Weather

In [249]:
# Set weather browser 
twitter_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(twitter_url)
weather_html = browser.html
weather_soup = BeautifulSoup(weather_html, 'lxml')
# print(weather_soup.prettify())                         

In [250]:
#Get latest tweet about weather from twitter link 
mars_weather = weather_soup.find_all(attrs={"data-testid":"tweet"})

print(mars_weather)


# tweet= [item['data-testid'] for item in weather_soup.find_all('ul', attrs={'data-testid' : True})]
# print(tweet)

[]


In [251]:
# Mars Facts

In [252]:
# Scrape second HTML table with pandas
mars_facts_url = 'https://space-facts.com/mars/'
mars_facts_table = pd.read_html(mars_facts_url)
mars_facts_table = mars_facts_table[1]

In [253]:
# print(mars_facts_table)

In [266]:
# Read table to pandas DataFrame
mars_facts_df = mars_facts_table
mars_facts_df.columns = ['Description', 'Value']
mars_facts_df

ValueError: Length mismatch: Expected axis has 3 elements, new values have 2 elements

In [255]:
# Convert to HTML table string
mars_facts_html = mars_facts_df.to_html()
print(mars_facts_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Mars - Earth Comparison</th>
      <th>Mars</th>
      <th>Earth</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Diameter:</td>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Moons:</td>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Distance from Sun:</td>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Length of Year:</td>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Temperature:</td>
      <td>-153 to 20 °C</td>
      <td>-88 to 58°C</td>
    </tr>
  </tbody>
</table>


In [256]:
# Mars Hemisphere Browser
hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars' 
browser.visit(hemisphere_url)
hemisphere_html = browser.html
hemisphere_soup = BeautifulSoup(hemisphere_html, 'lxml')

In [257]:
# Create list to hold list of links      
links = []  

# Create list to hold img urls
hemisphere_img_urls = []    

# Store base url in a variable for concatenation
base_url = "https://astrogeology.usgs.gov" 

result = hemisphere_soup.find('div', class_='result-list')
#print(result)

items = result.findAll('div', class_ = 'item')
#print(items)

#loop through divs
for item in items:
    # Create dictionary to append list of images
    hemisphere = {} 
    
    content = item.find('div', class_='description')
    
    title = content.find('h3').text
    hemisphere['title'] = title
    
    href = item.find('a', {"class":"itemLink product-item"})['href']
    links.append(base_url + href)
    
#loop through links
    for link in links:
        response = requests.get(link)
        hemisphere_soup = BeautifulSoup(response.text, 'lxml')

        img_src = hemisphere_soup.find("img", {"class":"wide-image"})['src']
        img_url = base_url + img_src
        hemisphere['img_url'] = img_url
        
    hemisphere_img_urls.append(hemisphere)
    
hemisphere_img_urls



[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]